In [ ]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics
from semisupervised_methods.clustream_and_label import  CluStreamMicroClusterWithLabel,CluserAndLabel
from river.forest.adaptive_random_forest import ARFClassifier
from semisupervised_methods.incremental_classifier import IncrementalClassifer
import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Airlines
from train_and_eval import train_and_evaluate
from functools import partial



In [5]:
# def FL(stream) -> bool:
#     """
#     #TODO: maybe change the implementaion
#     Indicate if the instance is labelled

#     Parameters
#     ----------
#     stream: list of tuple
#         list of instances x, and corresponing labels y (possible that it is None) #TODO: check if accurate assumption
#         The x must be a first element of a tuple

#     Returns
#     -------
#     list of tuples
#         list of labelled tuples

#     """
#     def hasLabel(instance):
#         '''
#         If the instance does not have label return None, otherwise it returns an instance
#         '''
#         if instance[3] is None:
#             return None
#         return instance

#     return list(filter(partial(is_not, None), list(map(partial(hasLabel), stream))))

In [ ]:
def get_most_frequent(predictions):
    return max(set(predictions), key = predictions.count)

In [ ]:
def update_performance_measures(predictions, true_label,B,metrics):
    interval = (len(predictions)-1)/B
    preds = list(predictions.values())
    for b in range(B+2):
        if b == 0:
            metrics[b].update(true_label,preds[0])
        elif b == B+1:
            metrics[b].update(true_label,preds[-1])
        else:
            y_pred = get_most_frequent(preds[int(1+interval*(b-1)):int((1+interval*b))])
            metrics[b].update(true_label,y_pred)
    

    

In [ ]:
def make_prediction_for_awaiting(h,cur_idx,P,L,K):
    # foreach instance in L add predictionin time cur_idx in P
    for idx, instance in L.items():
        #make prediction every K instances
        if abs(idx-cur_idx)%K == 0:
            P[idx][cur_idx] = h.predict_one(instance)

In [ ]:
def add_delay(stream, delay, no_delete_period,dataset_name,q):
    new_stream = []
    i = 0
    for idx_1, idx_2, x, y, in stream:
        if i < no_delete_period:
            new_stream.append((i, i, x, y))
            i = i+1
            continue
        if (i-no_delete_period) % delay == 0 and i-no_delete_period != 0:
            i += delay
        new_stream.append((i, i, x, None))
        new_stream.append((i+delay, i, x, y))
        i += 1
    new_stream.sort(key=lambda x: x[0])
    return StreamSection(f'{dataset_name}_delay_{q[0]}_{q[1]}', new_stream,False)


In [ ]:
def generate_streams(datastream,dataset_name,q,probas,with_delay,delay,warm_up_period):
    stream_set = []  # TODO chcek where this should be placed
    initial_stream = generate_stream_section(
        datastream, f'{dataset_name}_init_{q[0]}_{q[1]}', dataset_name, q[0], q[1])
    stream_set.append(initial_stream)
    if with_delay:
        initial_stream = add_delay(
            initial_stream.stream, delay, warm_up_period, dataset_name, q)
        dataset_name += '_delay'
        stream_set.append(initial_stream)
    
    for p in probas:
        ssl_stream = StreamSection(f'{dataset_name}_ssl_{p}_{q[0]}_{q[1]}', FU(
            initial_stream.stream, p, warm_up_period), False)
        lfs_stream = StreamSection(
            f'{dataset_name}_lfs_{p}_{q[0]}_{q[1]}', FL(ssl_stream.stream), True)
        stream_set.append(ssl_stream)
        stream_set.append(lfs_stream)
    return stream_set

In [ ]:
# K how many new labelled instances need to arrive before the new prediction is made
def train_and_evaluate(datastream, dataset_name, Q, probas, methods, methods_params, 
                       metric_fun, with_delay, K, B, delay, warm_up_period=10):
    '''
    Main evaluation and traing function
    '''
    results = {}
    for q in Q:
        # preparing streams part
        stream_set = generate_streams(datastream,dataset_name,q,probas,with_delay,delay,warm_up_period)

        # train and evaluation part
        results_for_q = {}
        for stream in stream_set:
            for mi, method in enumerate(methods):

                # initilaze method and variables
                m = method(**methods_params[mi])
                metrics = [metric_fun() for _ in range(B+2)]
                h = m
                L = {}
                P = {}

                for cur_idx, init_idx, x, y in stream.stream:
                    if(init_idx == 4565):
                        a = 0
                    # TODO: can it be in this place
                    make_prediction_for_awaiting(h, cur_idx, P, L, K)
                    # unlabelled instance
                    if y is None:
                        # add instnace and index
                        L[cur_idx] = x
                        P[cur_idx] = {}
                        P[cur_idx][cur_idx] = h.predict_one(x)
                        # TODO: think what to do if the method cannot deal with unlabelled
                        h = h.learn_one(x)

                    # labelled instance
                    else:
                        if cur_idx != init_idx and init_idx in P.keys():  # delayed label
                            P[init_idx][cur_idx] = h.predict_one(x)    
                            L.pop(init_idx)
                            
                            update_performance_measures(P[init_idx], y, B, metrics)
                        # TODO: probably need to implment a better option of evaluation-> for now test then train is used
                        else:
                            if m._timestamp > warm_up_period: # if in warmup period the prediction cannot be made
                                # if it was not delayed only last prediction exists
                                metrics[B+1].update(y, h.predict_one(x))

                        h = h.learn_one(x, y)
                results_for_q[stream.__name__] = metrics
            results[q] = results_for_q
    return results


In [ ]:
res = train_and_evaluate(led,dataset_name,Q,probas,methods,m_params,metrics.CohenKappa,True,10,5,100)
res

6
5
3
6
2
6
5
6
4
4
2
5
6
5
4
3
4
6
4
5
6
5
6


In [ ]:
led =  get_LED()
dataset_name = 'LED'
Q=[(0,10000)]
probas = [0.1,0.2,0.5]
clustream = CluserAndLabel
m_params = [{'train_period': 4,
 'n_macro_clusters': 5,
 'max_micro_clusters': 7,
 'micro_cluster_r_factor': 2,
 'time_window': 1000,
 'time_gap': 3,
 'seed': 0,
 'halflife': 0.4}]
methods = [clustream]

In [ ]:
initial_stream = generate_stream_section(led,f'{dataset_name}_init',dataset_name,0,10000)

In [ ]:
initial_stream = generate_stream_section(led,f'{dataset_name}_init_{q[0]}_{q[1]}',dataset_name,q[0],q[1])

In [ ]:
from river import cluster
from river import stream
# X = [ [1, 2],[1, 4],[1, 0],[-4, 0],[-4, 4]]

# Y = [1,1,2,3,1]
clustream = CluserAndLabel(4,
         max_micro_clusters=7,
         time_gap=3,
         seed=0,
         halflife=0.4 )
led_stream = datasets.synth.LED(123)
for i,(x,y) in enumerate(q.stream):
    if i <99:
        # if y ==7:
        #     print(clustream.return_microclusters() )
        if i %10 == 9:
            clustream = clustream.learn_one(x)
        else:
            clustream = clustream.learn_one(x = x,y = y)
        #print(clustream.sum_labels())

    else:
        print(x,clustream.predict_one(x),y)

clustream.return_microclusters() 

{0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 0, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1} 8 7
{0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0} 8 None
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 1, 16: 1, 17: 1, 18: 0, 19: 1, 20: 1, 21: 0, 22: 0, 23: 1} 8 None
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 1, 12: 0, 13: 1, 14: 1, 15: 0, 16: 0, 17: 1, 18: 0, 19: 1, 20: 1, 21: 0, 22: 1, 23: 0} 8 3
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 1, 15: 0, 16: 1, 17: 1, 18: 0, 19: 1, 20: 0, 21: 1, 22: 1, 23: 1} 8 2
{0: 0, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 1, 22: 1, 23: 0} 8 None
{0: 1, 1: 1, 

# DATASETS

In [ ]:
fligths = [(dict((i,x[i]) for i in x.keys() if i!='Delay'),x['Delay']) for x,_ in stream.iter_arff('../../datasets/airlines.arff.zip', compression='infer')]
lda = datasets.synth.LEDDrift(123,0,True,0)
electricity = [(dict((i,x[i]) for i in x.keys() if i!='class'),x['class']=='UP') for x,_ in stream.iter_arff('../../datasets/elecNormNew.arff.zip', compression='infer')]

In [ ]:
datasets.synth.LEDDrift()

In [ ]:
Q = [(0,1000),(1000,2000),(2000,3000),(3000,4000),(4000,5000)]
datastream = lda
P = [0.1,0.2,0.5,0.6]
dataset_name = 'LED'
stream_set = []

train_and_evaluate(datastream,dataset_name,Q,P)

In [ ]:
[x.__name__ for x in stream_set]

['Cover_Type_init_0_1000',
 'Cover_Type_ssl_0.1_0_1000',
 'Cover_Type_lfs_0.1_0_1000',
 'Cover_Type_ssl_0.2_0_1000',
 'Cover_Type_lfs_0.2_0_1000',
 'Cover_Type_ssl_0.5_0_1000',
 'Cover_Type_lfs_0.5_0_1000',
 'Cover_Type_ssl_0.6_0_1000',
 'Cover_Type_lfs_0.6_0_1000',
 'Cover_Type_init_1000_2000',
 'Cover_Type_ssl_0.1_1000_2000',
 'Cover_Type_lfs_0.1_1000_2000',
 'Cover_Type_ssl_0.2_1000_2000',
 'Cover_Type_lfs_0.2_1000_2000',
 'Cover_Type_ssl_0.5_1000_2000',
 'Cover_Type_lfs_0.5_1000_2000',
 'Cover_Type_ssl_0.6_1000_2000',
 'Cover_Type_lfs_0.6_1000_2000',
 'Cover_Type_init_2000_3000',
 'Cover_Type_ssl_0.1_2000_3000',
 'Cover_Type_lfs_0.1_2000_3000',
 'Cover_Type_ssl_0.2_2000_3000',
 'Cover_Type_lfs_0.2_2000_3000',
 'Cover_Type_ssl_0.5_2000_3000',
 'Cover_Type_lfs_0.5_2000_3000',
 'Cover_Type_ssl_0.6_2000_3000',
 'Cover_Type_lfs_0.6_2000_3000',
 'Cover_Type_init_3000_4000',
 'Cover_Type_ssl_0.1_3000_4000',
 'Cover_Type_lfs_0.1_3000_4000',
 'Cover_Type_ssl_0.2_3000_4000',
 'Cover_Type_lfs

In [ ]:
for q in stream_set:
    clustream = CluserAndLabel(4,
         max_micro_clusters=7,
         time_gap=3,
         seed=0,
         halflife=0.4 )
    for i,(x,y) in enumerate(q.stream):
        print(q.__name__)
        if i <990:
            clustream = clustream.learn_one(x = x,y = y)
            #print(clustream.sum_labels())

        else:
            print(clustream.predict_one(x),y)

    #clustream.return_microclusters() 

LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init

KeyboardInterrupt: 

In [ ]:
i

999

In [ ]:
hp = datasets.synth.RandomRBFDrift()
for i in hp.take(10):
    print(i)

({0: 0.4474707370607147, 1: 0.4112430647339043, 2: 0.5774887345532604, 3: 0.9345089494963857, 4: 0.625470816542423, 5: 0.9528893392318104, 6: 0.23390913685064327, 7: 0.2632706539603502, 8: 0.6487674488470405, 9: 0.3223666205525}, 0)
({0: 0.38615946253321076, 1: 0.40767762106825467, 2: 0.8820781268130979, 3: 1.0294262479340588, 4: 0.16964208338080689, 5: 0.4068516375090169, 6: 0.920811978379108, 7: 0.31470618207032774, 8: 0.2904818353582927, 9: 0.5318226376467426}, 1)
({0: 0.7407199640645749, 1: 0.675852803052017, 2: 0.09199640007669287, 3: 0.6597045689218738, 4: 0.9128605755224982, 5: 0.6715464514966379, 6: 0.5001450370818181, 7: 0.2822393064383389, 8: 0.9402458315878417, 9: 0.2657905405040588}, 0)
({0: 0.41429299227040856, 1: 0.40059196758918525, 2: 0.621787171747695, 3: 0.9122077031232282, 4: 0.6161816448475379, 5: 0.9673529227812119, 6: 0.24849079855787165, 7: 0.22698910907704506, 8: 0.6177538662843673, 9: 0.32919829542592816}, 0)
({0: 0.033234076182812905, 1: 0.5225248193493289, 2:

In [ ]:
model = tree.HoeffdingTreeClassifier(    grace_period=100,    delta=1e-5)

In [ ]:
metric = metrics.Accuracy()
evaluate.progressive_val_score(led_stream,model,metric)

Accuracy: 10.71%